# AdaBoost para regresión

<img src="https://raw.githubusercontent.com/fhernanb/fhernanb.github.io/master/my_docs/logo_unal_color.png" alt="drawing" width="200"/>

En este notebook se muestra un ejemplo de AdaBoost para regresión.

AdaBoost es un meta-estimador que comienza ajustando un regresor en el conjunto de datos original y luego ajusta copias adicionales del regresor en el mismo conjunto de datos, pero donde los pesos de las instancias se ajustan según el error de la predicción actual. De este modo, los regresores posteriores se centran más en los casos difíciles.

La función que vamos a usar es:

`sklearn.ensemble.AdaBoostRegressor(estimator=None, *, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None)`

Más detalles de la función pueden ser consultados [en este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor).

El objetivo del presente ejemplo es crear un modelo AdaBoost para estimar la variable $y$, número de satélites en la espalda de la cangreja, usando como covariables $weight$ y $width$. En la siguiente figura se muestra una figura ilustrativa del problema.

<img src="cangreja_con_un_satelite.png" alt="drawing" width="400"/>

In [1]:
from sklearn.ensemble import AdaBoostRegressor
import pandas as pd
import numpy
import matplotlib.pyplot as plt

In [2]:
datos = pd.read_csv('crabs.txt', sep=' ', header=0)
datos.head() # Para ver el encabezado de los datos

crab  y  weight  width  color  spine
1     1  8    3.05   28.3      2      3
2     2  0    1.55   22.5      3      3
3     3  9    2.30   26.0      1      1
4     4  0    2.10   24.8      3      3
5     5  4    2.60   26.0      3      3

In [3]:
datos.shape # Para ver la dimensión de la base de datos

(173, 6)

El código que se muestra abajo sirve para crear la matriz $\mathbf{X}$ y el vector $\mathbf{y}$ para entrenar el modelo.

In [4]:
X = datos[["weight", "width"]]
y = datos["y"]

mod = AdaBoostRegressor(random_state=0, n_estimators=50, loss='linear')
mod.fit(X, y)

AdaBoostRegressor(random_state=0)

Vamos a estimar el número de satélites y para dos hembras con la siguiente información:

- Hembra 1: weight=3.05 y width=28.3
- Hembre 2: weight=1.55 y width=22.5

In [5]:
new_X = {'weight': [3.05, 1.55],
         'width' : [28.3, 22.5]}
new_X = pd.DataFrame(new_X)
print(new_X)

print("\nThe predicted y's are:")
mod.predict(new_X)

   weight  width
0    3.05   28.3
1    1.55   22.5

The predicted y's are:


array([6.69090909, 2.28571429])

Vamos a calcular el coeficiente de correlación de Pearson entre $y$ y $\hat{y}$.

In [6]:
y_hat = mod.predict(X)

# Correlación entre y e y_hat
print(numpy.corrcoef(y, y_hat))

[[1.         0.61204655]
 [0.61204655 1.        ]]


Ahora vamos a calcular el $MSE=\frac{1}{n} \sum_{i=1}^{n} (y_i-\hat{y}_i)^2$.

In [7]:
def my_mse(y, y_hat):
    n = len(y)
    res = sum((y - y_hat)**2) / n
    return res

In [8]:
my_mse(y, y_hat)

6.992715398656685

Ahora vamos a crear un diagrama de dispersión entre $y$ y $\hat{y}$.

In [9]:
plt.scatter(x=y, y=y_hat, color='purple', alpha=0.15)
plt.xlabel('y: número de satélites')
plt.ylabel('y_hat: número estimado de satélites')
plt.title('Diagrama de dispersión para y_hat vs y');

## Explorando la función de pérdida

In [10]:
mod_lin = AdaBoostRegressor(random_state=0, n_estimators=50, loss='linear')
mod_lin.fit(X, y)

mod_sqr = AdaBoostRegressor(random_state=0, n_estimators=50, loss='square')
mod_sqr.fit(X, y)

mod_exp = AdaBoostRegressor(random_state=0, n_estimators=50, loss='exponential')
mod_exp.fit(X, y)

y_hat_lin = mod_lin.predict(X)
y_hat_sqr = mod_sqr.predict(X)
y_hat_exp = mod_exp.predict(X)

print('MSE con función de pérdida linear:', my_mse(y, y_hat_lin))
print('MSE con función de pérdida square:', my_mse(y, y_hat_sqr))
print('MSE con función de pérdida exponen:', my_mse(y, y_hat_exp))

MSE con función de pérdida linear: 6.992715398656685
MSE con función de pérdida square: 8.475895991611454
MSE con función de pérdida exponen: 7.766897608717595


## Explorando el número de estimador con loss fijo

In [11]:
mod_10 = AdaBoostRegressor(random_state=0, n_estimators=10, loss='linear')
mod_10.fit(X, y)

mod_30 = AdaBoostRegressor(random_state=0, n_estimators=30, loss='linear')
mod_30.fit(X, y)

mod_50 = AdaBoostRegressor(random_state=0, n_estimators=50, loss='linear')
mod_50.fit(X, y)

y_hat_10 = mod_10.predict(X)
y_hat_30 = mod_30.predict(X)
y_hat_50 = mod_50.predict(X)

print('MSE con n=10:', my_mse(y, y_hat_10))
print('MSE con n=30:', my_mse(y, y_hat_30))
print('MSE con n=50:', my_mse(y, y_hat_50))

MSE con n=10: 7.178992482432526
MSE con n=30: 6.992715398656685
MSE con n=50: 6.992715398656685
